In [1]:
import tkinter as tk
from tkinter import filedialog, simpledialog, messagebox
import json
import os
from PIL import Image, ImageTk


class ImageLabelingApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Etykietowanie zdjęć")

        self.image_folder = None
        self.image_files = []
        self.current_image_index = 0
        self.labels = {}  # Tabela etykiet
        self.current_label = None  # Aktualna etykieta
        self.rectangles = []  # Prostokąty do rysowania
        self.label_colors = {}  # Kolory dla etykiet

        self.canvas = tk.Canvas(self.root, width=800, height=800, bg="#f0f0f0")
        self.canvas.pack(side=tk.LEFT, padx=10, pady=10)

        self.control_frame = tk.Frame(self.root, bg="#e6f2ff", bd=2, relief=tk.RIDGE)
        self.control_frame.pack(side=tk.RIGHT, padx=20, pady=20, fill=tk.Y)

        tk.Label(self.control_frame, text="Etykiety", font=("Helvetica", 16, "bold"), bg="#e6f2ff").pack(pady=10)

        self.label_listbox = tk.Listbox(self.control_frame, selectmode=tk.SINGLE, height=10, font=("Helvetica", 14), bd=2, relief=tk.SUNKEN)
        self.label_listbox.pack(pady=10, padx=10, fill=tk.X)

        self.add_label_button = tk.Button(self.control_frame, text="Dodaj etykietę", command=self.add_new_label, font=("Helvetica", 14), width=20, height=2, bg="#cce7ff", activebackground="#99ccff", relief=tk.RAISED, bd=2)
        self.add_label_button.pack(pady=10)

        self.reset_button = tk.Button(self.control_frame, text="Resetuj zaznaczenie", command=self.reset_last_rectangle, font=("Helvetica", 14), width=20, height=2, bg="#cce7ff", activebackground="#99ccff", relief=tk.RAISED, bd=2)
        self.reset_button.pack(pady=10)

        self.update_label_listbox()

        self.image = None
        self.photo = None
        self.original_image = None
        self.zoom_level = 1.0

        self.root.bind("<space>", self.next_image)
        # Zmiana klawiszy dla etykiet 1-9
        for i in range(1, 10):
            self.root.bind(f"<Key-{i}>", lambda event, i=i: self.set_label(str(i)))

        self.root.bind("<Button-1>", self.start_drawing)
        self.root.bind("<B1-Motion>", self.draw_rectangle)
        self.root.bind("<ButtonRelease-1>", self.end_drawing)

        # Dodajemy domyślne etykiety
        self.add_default_labels()

        self.start_screen()

    def add_default_labels(self):
        default_labels = [
            "Apple", "Avocado", "Kiwi", "Lemon", "Orange",
            "Pear", "Pineapple", "Pomegranate","Strawberry"
        ]
        for label in default_labels:
            label_id = str(len(self.labels) + 1)
            self.labels[label_id] = label
            self.label_colors[label_id] = self.random_color()  # Przypisanie koloru do etykiety
        self.update_label_listbox()

    def random_color(self):
        import random
        return f"#{random.randint(0, 0xFFFFFF):06x}"  # Generuje losowy kolor w formacie hex

    def update_label_listbox(self):
        self.label_listbox.delete(0, tk.END)
        for label in self.labels.values():
            self.label_listbox.insert(tk.END, label)

    def add_new_label(self):
        new_label = simpledialog.askstring("Nowa etykieta", "Podaj nazwę nowej etykiety:")
        if new_label:
            label_id = str(len(self.labels) + 1)
            self.labels[label_id] = new_label
            self.label_colors[label_id] = self.random_color()  # Przypisanie koloru do etykiety
            self.update_label_listbox()

    def start_screen(self):
        folder = filedialog.askdirectory(title="Wybierz folder ze zdjęciami")
        if not folder:
            self.root.quit()
            return

        self.image_folder = folder
        self.image_files = [f for f in os.listdir(folder) if f.lower().endswith(('jpg', 'jpeg', 'png', 'bmp', 'gif'))]
        self.image_files.sort()
        if not self.image_files:
            print("Brak obrazów w folderze.")
            self.root.quit()
            return

        self.load_image()

    def load_image(self):
        if self.current_image_index >= len(self.image_files):
            print("Koniec zdjęć!")
            messagebox.showinfo("Koniec zdjęć", "To jest ostatnie zdjęcie.")
            self.root.quit()
            return

        image_path = os.path.join(self.image_folder, self.image_files[self.current_image_index])
        self.original_image = Image.open(image_path)
        self.original_width, self.original_height = self.original_image.size
        self.image = self.original_image.resize((800, 800))
        self.photo = ImageTk.PhotoImage(self.image)

        self.canvas.delete("all")
        self.canvas.create_image(400, 400, image=self.photo)

        # Rysowanie prostokątów z zapisanymi koordynatami
        for rect in self.rectangles:
            color = self.label_colors.get(str(rect[4]), "#000000")  # Kolor przypisany do labelu
            self.canvas.create_rectangle(rect[0], rect[1], rect[2], rect[3], outline=color, width=2)

    def next_image(self, event):
        self.current_image_index += 1
        self.rectangles = []  # Resetowanie prostokątów
        self.load_image()

    def set_label(self, label_id):
        if label_id in self.labels:
            self.current_label = label_id
            self.highlight_selected_label()  # Podświetlanie wybranego labelu

    def highlight_selected_label(self):
        """Podświetlanie wybranego labelu w menu."""
        self.label_listbox.selection_clear(0, tk.END)
        self.label_listbox.activate(int(self.current_label) - 1)
        self.label_listbox.selection_set(int(self.current_label) - 1)

    def start_drawing(self, event):
        if self.current_label is None:
            return  # Nie rysuj, jeśli nie ma wybranej etykiety
        self.start_x = self.canvas.canvasx(event.x)
        self.start_y = self.canvas.canvasy(event.y)

    def draw_rectangle(self, event):
        if self.current_label is None:
            return  # Nie rysuj, jeśli nie ma wybranej etykiety
        self.end_x = self.canvas.canvasx(event.x)
        self.end_y = self.canvas.canvasy(event.y)

        # Ograniczenie rysowania kwadratów tylko w obrębie zdjęcia
        if self.end_x > 800: self.end_x = 800
        if self.end_y > 800: self.end_y = 800
        if self.start_x > 800: self.start_x = 800
        if self.start_y > 800: self.start_y = 800

        self.canvas.delete("temp")
        self.canvas.create_rectangle(self.start_x, self.start_y, self.end_x, self.end_y, outline="blue", width=2, tags="temp")

    def end_drawing(self, event):
        if self.current_label is None:
            return  # Nie rysuj, jeśli nie ma wybranej etykiety

        end_x = self.canvas.canvasx(event.x)
        end_y = self.canvas.canvasy(event.y)

        # Koordynaty muszą być w oryginalnej wielkości zdjęcia
        scale_x = self.original_width / 800
        scale_y = self.original_height / 800

        original_start_x = int(min(self.start_x, end_x) * scale_x)
        original_start_y = int(min(self.start_y, end_y) * scale_y)
        original_end_x = int(max(self.start_x, end_x) * scale_x)
        original_end_y = int(max(self.start_y, end_y) * scale_y)

        color = self.label_colors.get(self.current_label, "#000000")
        self.rectangles.append((original_start_x, original_start_y, original_end_x, original_end_y, self.labels[self.current_label]))  # Zapisujemy nazwę etykiety, nie ID
        self.canvas.delete("temp")
        self.canvas.create_rectangle(self.start_x, self.start_y, end_x, end_y, outline=color, width=2)

        self.save_annotations()

    def reset_last_rectangle(self):
        """Usuwa ostatnio narysowany prostokąt."""
        if self.rectangles:
            self.rectangles.pop()
            self.load_image()
        else:
            messagebox.showinfo("Brak zaznaczeń", "Nie ma zaznaczeń do usunięcia.")

    def save_annotations(self):
        annotations_file = os.path.join(self.image_folder, 'annotations.json')

        # Upewnijmy się, że plik JSON już istnieje
        if os.path.exists(annotations_file):
            with open(annotations_file, 'r') as file:
                data = json.load(file)
        else:
            data = []

        # Szukamy istniejącego zdjęcia
        existing_image = next((item for item in data if item['filename'] == self.image_files[self.current_image_index]), None)

        if existing_image:
            existing_image['labels'] = self.get_current_labels()  # Aktualizujemy dane
        else:
            data.append({
                "filename": self.image_files[self.current_image_index],
                "labels": self.get_current_labels()
            })

        # Zapisujemy dane do pliku
        with open(annotations_file, 'w') as file:
            json.dump(data, file, indent=4)

    def get_current_labels(self):
        # Pobieramy aktualne etykiety i koordynaty
        labels = []
        for rect in self.rectangles:
            label = rect[4]
            coords = [rect[0], rect[1], rect[2], rect[3]]
            label_found = False
            for label_data in labels:
                if label_data['label'] == label:
                    label_data['coordinates'].append(coords)
                    label_found = True
                    break
            if not label_found:
                labels.append({
                    "label": label,
                    "coordinates": [coords]
                })
        return labels


def main():
    root = tk.Tk()
    app = ImageLabelingApp(root)
    root.mainloop()


if __name__ == "__main__":
    main()

TclError: no display name and no $DISPLAY environment variable